In [ ]:
from yelpapi import YelpAPI
import tweepy
import json
import requests
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pprint import pprint
# Twitter key

TWITTER_KEY = "YOUR TWITTER_KEY"
TWITTER_KEY_SECRET = "YOUR TWITTER_KEY_SECRET"
TWITTER_ACCESS_TOKEN = "YOUR TWITTER_ACCESS_TOKEN"
TWITTER_ACCESS_TOKEN_SECRET = " YOUR TWITTER_ACCESS_TOKEN_SECRET"
# yelp keys
client_id = 'YOUR YELP CLIENT_ID'
api_key = 'YOUR YELP API_KEY'


yelp_api = YelpAPI(api_key)

auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_KEY_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
tweepy_api = tweepy.API(auth, wait_on_rate_limit=True
                        , wait_on_rate_limit_notify=True
                       )

In [ ]:
# Step 1: Generate the geolocations from yelp api. In this step, 3 datasets are generaed
'''
dataset 1: Yelp rating above 4
dataset 2 : Yelp rating between 3 and 4
dataset 3 : yelp rating below 3
'''

# https://www.yelp.com/developers/documentation/v3/business_search

import json
from pprint import pprint

term = ['food','drinks']
location = ['orlando','las vegas']
search_limit = 5
sortbyProperty = 'review_count'

#  This section retrives dataset of rating above 4

no_of_records = 0
offset_value  = 0
while(no_of_records < 15 and offset_value < 950):
    with open('yelpoutabove4t.txt','w') as outfile:
        for loc in location:
            for t in term:
                response = yelp_api.search_query(
                                                    term = t,
                                                    location = loc,
                                                    limit = search_limit,
                                                    offset = offset_value,
                                                    sort_by = sortbyProperty
                                                )
                for r in response['businesses']:
                    if not r['is_closed'] and r['rating'] >= 4: #true - permanently closed, we don't focu on closed ones
                        data = {}
                        data['restaurants']= []
                        data['restaurants'].append({
                                'keyword':t,
                                'name' : r['name'],
                                'city': r['location']['city'],
                                'coordinates':r['coordinates'],
                                'review_count':r['review_count'],
                                'rating':r['rating']
                        })

                        print(r['name'] + ' ,rating : ' + str(r['rating']))
    #                     pprint(data)
                        json.dump(data,outfile)
                        outfile.write('\n')
                        no_of_records += 1
    offset_value = offset_value + 50
print(80 * '*')
print('Number of restaurants of rating above 4 in yelp: '+ str(no_of_records))
print(80 * '*')

# #  This section retrives dataset of rating between 3 and 4

no_of_records = 0
offset_value  = 0

with open('yelpoutbetween3and4t.txt','w') as outfile:
    while(no_of_records < 15 and offset_value < 950):
        for loc in location:
            for t in term:
                response = yelp_api.search_query(
                                                    term = t,
                                                    location = loc,
                                                    limit = search_limit,
                                                    offset = offset_value,
                                                    sort_by = sortbyProperty
                                                )
                for r in response['businesses']:
                    if not r['is_closed'] and r['rating'] >= 3 and r['rating'] < 4: #true - permanently closed, we don't focu on closed ones
                        data = {}
                        data['restaurants']= []
                        data['restaurants'].append({
                                'keyword':t,
                                'name' : r['name'],
                                'city': r['location']['city'],
                                'coordinates':r['coordinates'],
                                'review_count':r['review_count'],
                                'rating':r['rating']
                        })

                        print(r['name'] + ' ,rating : ' + str(r['rating']))
    #                     pprint(data)
                        json.dump(data,outfile)
                        outfile.write('\n')
                        no_of_records += 1
        offset_value = offset_value + 50
print(80 * '*')
print('Number of restaurants of rating between 3 and 4 in yelp: '+ str(no_of_records))
print(80 * '*')

# #  This section retrives dataset of rating below 3

term = ['food','drinks','coffee']
location = ['orlando','las vegas']
no_of_records = 0
offset_value  = 500
search_limt = 50
sortbyProperty = 'review_count'
# itr = 1

#     print('iteration: '+ str(itr) + ' ,offset:  '+ str(offset_value))
with open('yelpoutbelow3t.txt','w') as outfile:
    while(no_of_records < 15 and offset_value < 950):
        for loc in location:
            for t in term:
                response = yelp_api.search_query(
                                                    term = t,
                                                    location = loc,
                                                    limit = search_limit,
                                                    offset = offset_value,
                                                    sort_by = sortbyProperty
                                                )
                for r in response['businesses']:
                    if not r['is_closed'] and r['rating'] < 3: #true - permanently closed, we don't focu on closed ones
                        data = {}
                        data['restaurants']= []
                        data['restaurants'].append({
                                'keyword':t,
                                'name' : r['name'],
                                'city': r['location']['city'],
                                'coordinates':r['coordinates'],
                                'review_count':r['review_count'],
                                'rating':r['rating']
                        })

                        print(r['name'] + ' ,rating : ' + str(r['rating']))
    #                     pprint(data)
                        json.dump(data,outfile)
                        outfile.write('\n')
                        no_of_records += 1
#         itr = itr + 1
        offset_value = offset_value + 50
print(80 * '*')
print('Number of restaurants of rating below 3: '+ str(no_of_records))
print(80 * '*')
                    

In [ ]:
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets
# Step 2: generate tweets for the geolocation derieved from the above step
TWITTER_RADIUS = ".1mi"
max_count = 50
no_of_records = 0

# start_date = datetime.datetime(2020, 9, 20, 00, 00, 00)
# end_date = datetime.datetime(2020, 9, 20, 15, 00, 00)
# Twitter data for input file 1

with open('twitteroutabove4t.txt','w') as outfile:
    f = open("yelpoutabove4t.txt","r")
    lines = f.readlines()
    for line in lines:  
        d = json.loads(line)
#         pprint(line)
        for rec in [data for data in d.values()][0]:
            name = rec['name']
            lat = rec['coordinates']['latitude']
            lon = rec['coordinates']['longitude']
            print ('*'*80)
            print(name)
            print ('*'*80)
            tweet_text_list = []
            for single_tweet in tweepy.Cursor(tweepy_api.search,
#                                                q=name,
                                               count=100,
                                               geocode="{},{},{}".format(lat,lon,TWITTER_RADIUS)
                                               ,lang="en"
                                               ,result_type = "mixed" 
#                                                ,since_id = 1331090395879649280
#                                                ,max_id = 1332436488718585857
#                                                ,since = '2020-11-24'
#                                                ,until = '2020-11-25'
                                            ).items(max_count):
#                 print("-->" + single_tweet.text)
#                 print("****** " + str(single_tweet.id) +" ********")
            
#                 print('**** ' + str(single_tweet.created_at) + ' ******* ' + str(sinceid))
#                 sinceid = sinceid + 1
                tweet_text_list.append(single_tweet.text)
            
            data = {}
            data['restaurants']= []
            data['restaurants'].append({                
                                        "name": name,
                                        "coordinates": {
                                                        "latitude":lat,
                                                        "longitude":lon
                                                        },
                                        "city":d['restaurants'][0]['city'],
                                        "review_count": d['restaurants'][0]['review_count'],
                                        "yelp_rating": d['restaurants'][0]['rating'],
                                        "tweet_data": tweet_text_list
                                   })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
            print(no_of_records)
            print('-'*80)    
        
    f.close()
    
    
# Twitter data for input file 2

with open('twitteroutbetween3and4t.txt','w') as outfile:
    f = open("yelpoutbetween3and4t.txt","r")
    lines = f.readlines()
    for line in lines:  
        d = json.loads(line)
#         pprint(line)
        for rec in [data for data in d.values()][0]:
            name = rec['name']
            lat = rec['coordinates']['latitude']
            lon = rec['coordinates']['longitude']
            print ('*'*80)
            print(name)
            print ('*'*80)
            tweet_text_list = []
            for single_tweet in tweepy.Cursor(tweepy_api.search,
#                                                q=name,
                                               count=100,
                                               geocode="{},{},{}".format(lat,lon,TWITTER_RADIUS)
                                               ,lang="en"
                                               ,result_type = "mixed" 
#                                                ,since_id = 1331090395879649280
#                                                ,max_id = 1332436488718585857
#                                                ,since = '2020-11-24'
#                                                ,until = '2020-11-25'
                                            ).items(max_count):
#                 print("-->" + single_tweet.text)
#                 print("****** " + str(single_tweet.id) +" ********")
            
#                 print('**** ' + str(single_tweet.created_at) + ' ******* ' + str(sinceid))
#                 sinceid = sinceid + 1
                tweet_text_list.append(single_tweet.text)
            
            data = {}
            data['restaurants']= []
            data['restaurants'].append({                
                                        "name": name,
                                        "coordinates": {
                                                        "latitude":lat,
                                                        "longitude":lon
                                                        },
                                        "city":d['restaurants'][0]['city'],
                                        "review_count": d['restaurants'][0]['review_count'],
                                        "yelp_rating": d['restaurants'][0]['rating'],
                                        "tweet_data": tweet_text_list
                                   })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
            print(no_of_records)
            print('-'*80)    
        
    f.close()
    

# Twitter data for input file 3

with open('twitteroutbelow3t.txt','w') as outfile:
    f = open("yelpoutbelow3t.txt","r")
    lines = f.readlines()
    for line in lines:  
        d = json.loads(line)
#         pprint(line)
        for rec in [data for data in d.values()][0]:
            name = rec['name']
            lat = rec['coordinates']['latitude']
            lon = rec['coordinates']['longitude']
            print ('*'*80)
            print(name)
            print ('*'*80)
            tweet_text_list = []
            for single_tweet in tweepy.Cursor(tweepy_api.search,
#                                                q=name,
                                               count=100,
                                               geocode="{},{},{}".format(lat,lon,TWITTER_RADIUS)
                                               ,lang="en"
                                               ,result_type = "mixed" 
#                                                ,since_id = 1331090395879649280
#                                                ,max_id = 1332436488718585857
#                                                ,since = '2020-11-24'
#                                                ,until = '2020-11-25'
                                            ).items(max_count):
#                 print("-->" + single_tweet.text)
#                 print("****** " + str(single_tweet.id) +" ********")
            
#                 print('**** ' + str(single_tweet.created_at) + ' ******* ' + str(sinceid))
#                 sinceid = sinceid + 1
                tweet_text_list.append(single_tweet.text)
            
            data = {}
            data['restaurants']= []
            data['restaurants'].append({                
                                        "name": name,
                                        "coordinates": {
                                                        "latitude":lat,
                                                        "longitude":lon
                                                        },
                                        "city":d['restaurants'][0]['city'],
                                        "review_count": d['restaurants'][0]['review_count'],
                                        "yelp_rating": d['restaurants'][0]['rating'],
                                        "tweet_data": tweet_text_list
                                   })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
            print(no_of_records)
            print('-'*80)    
        
    f.close()
    
print(no_of_records)

In [ ]:
from pprint import pprint
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = "N/A"
analyzer = SentimentIntensityAnalyzer()
no_of_records = 0

# Analysing the Tweets corresponding to dataset1

with open('finaloutabove4t.txt','w') as outfile:
    f = open("twitteroutabove4t.txt","r")
    lines = f.readlines()
    for line in lines:
#         no_of_records += 1
        d = json.loads(line)
        for rec in [data for data in d.values()][0]:
#             pprint(rec['tweet_data'])
            comp_score = 0
            text_cnt = 0
            for text in rec['tweet_data']:
                vs = analyzer.polarity_scores(text)
                comp_score += vs['compound']
                text_cnt += 1
                cum_comp_score = comp_score/text_cnt
                if cum_comp_score >= 0.05:
                    sentiment = "POSITIVE"
                elif (cum_comp_score < 0.05) and (cum_comp_score > -0.05):
                    sentiment = "NEUTRAL"
                else:
                    sentiment = "NEGATIVE"
            if text_cnt ==0:
                    sentiment = "N/A"
            data = {}
            data['restaurants']= []
            data['restaurants'].append({
                                        'name' : rec['name'],
                                        'coordinates':rec['coordinates'],
                                        'city':rec['city'],
                                        'yelp_review_count':rec['review_count'],
                                        'tweet_cnt':text_cnt,
                                        'yelp_rating':rec['yelp_rating'],
                                        'vader_polarity':sentiment
                                    })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
    f.close()
print(no_of_records)

# Analysing the Tweets corresponding to dataset 2

with open('finaloutbetween3and4t.txt','w') as outfile:
    f = open("twitteroutbetween3and4t.txt","r")
    lines = f.readlines()
    for line in lines:
#         no_of_records += 1
        d = json.loads(line)
        for rec in [data for data in d.values()][0]:
#             pprint(rec['tweet_data'])
            comp_score = 0
            text_cnt = 0
            for text in rec['tweet_data']:
                vs = analyzer.polarity_scores(text)
                comp_score += vs['compound']
                text_cnt += 1
                cum_comp_score = comp_score/text_cnt
                if cum_comp_score >= 0.05:
                    sentiment = "POSITIVE"
                elif (cum_comp_score < 0.05) and (cum_comp_score > -0.05):
                    sentiment = "NEUTRAL"
                else:
                    sentiment = "NEGATIVE"
            if text_cnt ==0:
                    sentiment = "N/A"
            data = {}
            data['restaurants']= []
            data['restaurants'].append({
                                        'name' : rec['name'],
                                        'coordinates':rec['coordinates'],
                                        'city':rec['city'],
                                        'yelp_review_count':rec['review_count'],
                                        'tweet_cnt':text_cnt,
                                        'yelp_rating':rec['yelp_rating'],
                                        'vader_polarity':sentiment
                                    })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
    f.close()
print(no_of_records)

# Analysing the Tweets corresponding to dataset 3

with open('finaloutbelow3t.txt','w') as outfile:
    f = open("twitteroutbelow3t.txt","r")
    lines = f.readlines()
    for line in lines:
#         no_of_records += 1
        d = json.loads(line)
        for rec in [data for data in d.values()][0]:
#             pprint(rec['tweet_data'])
            comp_score = 0
            text_cnt = 0
            for text in rec['tweet_data']:
                vs = analyzer.polarity_scores(text)
                comp_score += vs['compound']
                text_cnt += 1
                cum_comp_score = comp_score/text_cnt
                if cum_comp_score >= 0.05:
                    sentiment = "POSITIVE"
                elif (cum_comp_score < 0.05) and (cum_comp_score > -0.05):
                    sentiment = "NEUTRAL"
                else:
                    sentiment = "NEGATIVE"
            if text_cnt ==0:
                    sentiment = "N/A"
            data = {}
            data['restaurants']= []
            data['restaurants'].append({
                                        'name' : rec['name'],
                                        'coordinates':rec['coordinates'],
                                        'city':rec['city'],
                                        'yelp_review_count':rec['review_count'],
                                        'tweet_cnt':text_cnt,
                                        'yelp_rating':rec['yelp_rating'],
                                        'vader_polarity':sentiment
                                    })
            pprint(data)
            json.dump(data,outfile)
            outfile.write('\n')
            no_of_records += 1
    f.close()
print(no_of_records)

In [ ]:
# analysis of the polarity

filenames = ["finaloutabove4t.txt","finaloutbetween3and4t.txt","finaloutbelow3t.txt"]
for file in filenames:
    f = open(file,"r")
    lines = f.readlines()
    pos_count = 0;
    neg_count = 0;
    neutral_count = 0;
    NA_count = 0;
    total_records = 0;
    for line in lines:
        d = json.loads(line)
        for rec in [data for data in d.values()][0]:
            total_records += 1
            if rec['vader_polarity'] == 'POSITIVE':
                pos_count += 1
            elif rec['vader_polarity'] == 'NEGATIVE':
                neg_count += 1
            elif rec['vader_polarity'] == 'NEUTRAL':
                neutral_count += 1
            else:
                NA_count +=1   
    f.close()
    print('FileName : ' + file)
    print(80*'-')
    print("Total Records = {}".format(total_records))
    print("Positive Sentiment = {}".format(pos_count))
    print("Negative Sentiment = {}".format(neg_count))
    print("Neutral Sentiment = {}".format(neutral_count))
    print("Tweets not available for {} locations".format(NA_count))
    print(80*'*')

In [ ]:
# To count the Total number of Tweets from twitteroutabove4, twitteroutbetween3and4 and twitteroutbelow3

from pprint import pprint
import json
filenames = ["twitteroutabove4t.txt","twitteroutbetween3and4t.txt","twitteroutbelow3t.txt"]
total_tweet_count = 0
for file in filenames:
    f=open(file,'r')
    lines = f.readlines()
    for line in lines:
        d = json.loads(line)
        for rec in [data for data in d.values()][0]:
            print('Name:' + rec['name'] + ' ,Tweet Count: ' + str(len(rec['tweet_data'])) )
            total_tweet_count = total_tweet_count + len(rec['tweet_data'])
    f.close()
print(total_tweet_count)
